# K-gram indices

This week the exercise focuses on k-gram indices, and two uses for them: (i) wildcard queries (`some*e`) and (ii) spell correction (`smoetime` → `sometime`).

In [1]:
import sys
sys.path.append("../../")

In [2]:
import re
import glob
from textutils import tokenize_document
from queryparser import parse_query, ast_has_operation, process_ast, Operation, ParseException

The provided window function returns sliding-n-window subsequences of the input sequence:

In [3]:
def window(seq, n):
    for i in range(len(seq) - n + 1):
        yield seq[i:i+n]
print(list(window([2, 3, 4, 5, 6], 2)))
print(list(window("astring", 3)))

[[2, 3], [3, 4], [4, 5], [5, 6]]
['ast', 'str', 'tri', 'rin', 'ing']


In [4]:
# Map document titles to document ids
documents = {}
# A running counter for assigning numerical IDs to documents'''
docid_counter = 1
# The posting lists
the_index = {}
# K-gram index
kgram_index = dict()
# The K in k-gram
K = 3

Note that from this week onwards the posting lists will be a python `set`: this way we can use its built in deduplication, `&` (intersection), `|` (union), and `-` (difference):

In [5]:
example_set = {3, 4}
example_set.add(6)
example_set.add(4)
print(example_set)
print(example_set & {4, 6, 7})
print(example_set | {4, 6, 7})
print(example_set - {4, 6, 7})

{3, 4, 6}
{4, 6}
{3, 4, 6, 7}
{3}


Add documents to the index, and words to the k-gram index. Make sure the posting lists are python `set`s.

For the k-gram index you will want to bracket each term in the vocabulary of
your standard inverted index with dollar signs, which are used as word
boundary markers and are important to process queries where the wildcard is at
the beginning or at the end of a term.

In [6]:
# wipe existing data
documents = {}
docid_counter = 1
the_index = {}
kgram_index = dict()

for doc in glob.glob('../../shared/corpus/*.txt'):
    docid = docid_counter
    documents[docid] = doc
    docid_counter += 1
    print("Added document %s with id %d" % (doc, docid))
    for word in tokenize_document(doc):
        the_index.setdefault(word, set()).add(docid)

for word in the_index.keys():
    ### <assignment>
    for gram in window("$" + word + "$", K):
        if gram != "$$":
            kgram_index.setdefault(gram, set()).add(word)
    ### </assignment>

Added document ../../shared/corpus\alls_well_that_ends_well.txt with id 1
Added document ../../shared/corpus\as_you_like_it.txt with id 2
Added document ../../shared/corpus\a_lovers_complaint.txt with id 3
Added document ../../shared/corpus\a_midsummer_nights_dream.txt with id 4
Added document ../../shared/corpus\cymbeline.txt with id 5
Added document ../../shared/corpus\king_henry_the_eighth.txt with id 6
Added document ../../shared/corpus\king_john.txt with id 7
Added document ../../shared/corpus\king_richard_the_second.txt with id 8
Added document ../../shared/corpus\king_richard_the_third.txt with id 9
Added document ../../shared/corpus\loves_labours_lost.txt with id 10
Added document ../../shared/corpus\measure_for_measure.txt with id 11
Added document ../../shared/corpus\much_ado_about_nothing.txt with id 12
Added document ../../shared/corpus\pericles_prince_of_tyre.txt with id 13
Added document ../../shared/corpus\the_comedy_of_errors.txt with id 14
Added document ../../shared/c

Here's a test to check the k-gram index is valid:

In [7]:
assert(kgram_index['$ze'] == {'zeal', 'zeale', 'zealous', 'zeals', 'zeal—', 'zed', 'zenith', 'zephyrs'})

## Wildcard queries

Write a function that takes a wildcard query term (`'some*e'`) and returns all the keys to query in the k-gram index.

In [8]:
def wildcard_parse(q):
    ### <assignment>
    if q[0] != '*':
        q = "$" + q
    if q[-1] != '*':
        q = q + "$"
    grams = [t for t in window(q, K) if "*" not in t]
    return grams
    ### </assignment>

Tests:

In [9]:
assert(wildcard_parse('some*e') == ['$so', 'som', 'ome'])
assert(wildcard_parse('*where') == ['whe', 'her', 'ere', 're$'])
assert(wildcard_parse('some*ere') == ['$so', 'som', 'ome', 'ere', 're$'])

Implement querying the k-gram index for matching words, given a certain wildcard query `q`. Here's an high-level description for the implementation:

  * parse the wildcard query with `wildcard_parse`;
  * check whether any of the returned kgrams are not in the `kgram_index`: if that's the case, there are no matches for this wildcard query, and and empty set should be returned;
  * (optionally) compute the number of matches for each kgram and order them from smaller to larger;
  * intersect the word matches for each kgram;
  * perform post-processing to exclude false positives (we provide some code that converts the wildcard query in a python regex that only matches valid words);
  * return the set of matching words.

In [10]:
def kgram_wildcard_query(q):
    grams = wildcard_parse(q)
    ### <assignment>
    for t in grams:
        if t not in kgram_index:
            print("NOTE: no wildcard matches for '%s'" % q)
            return set()
    words = [kgram_index[t] for t in grams]
    freqs = sorted([ (p, len(p)) for p in words ], key=lambda x: x[1])
    result = freqs[0][0]
    del freqs[0]
    while freqs != []:
        result = result & freqs[0][0]
        del freqs[0]
    post_filter = re.compile("^" + q.replace("*", "\\w*") + "$")
    res = {r for r in result if post_filter.match(r) is not None}
    print("NOTE: wildcard matches for '%s': %s" % (q, res))
    return res
    ### </assignment>

Don't forget to perform post-processing to exclude false positives. Tests:

In [11]:
assert(kgram_wildcard_query('some*e') == {'someone', 'somewhere', 'sometime'})
assert(kgram_wildcard_query('*where') == {'otherwhere', 'everywhere', 'nowhere', 'elsewhere', 'anywhere', 'where', 'somewhere'})
assert(kgram_wildcard_query('rend*') == {'rend', 'render', 'renderd', 'rendered', 'renders', 'rendezvous', 'rendred', 'rendring'})
len(kgram_wildcard_query('man*'))

NOTE: wildcard matches for 'some*e': {'somewhere', 'sometime', 'someone'}
NOTE: wildcard matches for '*where': {'elsewhere', 'otherwhere', 'everywhere', 'somewhere', 'nowhere', 'anywhere', 'where'}
NOTE: wildcard matches for 'rend*': {'rendezvous', 'rendered', 'render', 'renders', 'renderd', 'rendring', 'rend', 'rendred'}
NOTE: wildcard matches for 'man*': {'mannadge', 'managed', 'manyheaded', 'manycolourd', 'managing', 'mandrakes', 'manage', 'manorhouse', 'manlike', 'manatarms', 'mannerly', 'manwhose', 'manner', 'manifoldly', 'mankinde', 'manhoods', 'manual', 'manmy', 'manager', 'manycoloured', 'mane', 'manes', 'manfully', 'manifests', 'manifold', 'manacle', 'mannersas', 'mankind', 'mannd', 'manifest', 'manacles', 'man', 'manofwar', 'manus', 'manywhom', 'manakin', 'mandrake', 'manhood', 'manifested', 'manured', 'mantled', 'mans', 'mangy', 'manchild', 'mangled', 'mannish', 'manors', 'mantles', 'many', 'mansions', 'manure', 'manners', 'mandragora', 'manfew', 'manly', 'mansion', 'manmons

69

We provide a generic implementation of `execute_query`. Note that intersection/union operations are implemented with Python's sets. This implementation will invoke `kgram_wildcard_query` when it encounters a wildcard query during preprocessing. It will replace the wildcard term with a disjunction of the matching words (`some*e` → `someone OR somewhere OR sometime`).

In [12]:
# temporary empty implementation of `spellcorrect`: this will be relevant later
def spellcorrect(arg):
    return None

def execute_query(query):
    def negate(postings):
        return set(documents.keys()) - postings

    try:
        ast = parse_query(query)
    except ParseException as e:
        print("Failed to parse query '%s'\n" % query, e)
        return None

    flat = process_ast(ast)

    def preprocess_query_tree(tree):
        if tree.op == 'LOOKUP':
            tree.op = 'AND'
        new_args = []
        for arg in tree.args:
            if isinstance(arg, Operation):
                new_args.append(preprocess_query_tree(arg))
            elif "*" in arg:
                kgram_matches = kgram_wildcard_query(arg)
                if len(kgram_matches) == 0:
                    print("NOTE: spell-correcting term '%s' because no document contains it" % arg)
                elif arg.startswith('-'):
                    not_op = Operation('NOT', [
                        Operation('OR', list(kgram_matches)),])
                    new_args.append(not_op)
                elif tree.op == 'OR':
                    new_args.extend(kgram_matches)
                else:
                    new_args.append(Operation('OR', kgram_matches))
            else:
                if not arg.startswith('-') and arg not in the_index:
                    print("NOTE: spell-correcting term '%s' because no document contains it" % arg)
                    res = spellcorrect(arg)
                    # NOTE: spellcorrect is the second part of this exercise, and will return None until implemented
                    if res != None and res != []:
                        print("NOTE: spell-corrections with Jaccard_threshold for '%s': %s" % (arg, res))
                        new_args.append(Operation('OR', list(res)))
                else:
                    new_args.append(arg)
        tree.args = new_args

    preprocess_query_tree(flat)

    def execute_query_tree(tree):
        result = set()
        if tree.op == 'AND':
            result = set(documents.keys())
        for arg in tree.args:
            if isinstance(arg, Operation):
                temp = execute_query_tree(arg)
            elif arg.startswith('-'):
                temp = negate(the_index[arg[1:]])
            else:
                temp = the_index[arg]

            if tree.op == 'OR':
                result = result | temp
            elif tree.op == 'AND':
                result = result & temp
            elif tree.op == 'NOT':
                assert(len(tree.args) == 1)
                result = negate(temp)
        return result

    return execute_query_tree(flat)

In [13]:
# adapted to windows lexicographic ordering!
assert(execute_query('some*e') == {1, 3, 4, 5, 8, 10, 11, 12, 13, 14, 15, 19, 21, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44})
assert(execute_query('some*e AND Romeo') == {38})

NOTE: wildcard matches for 'some*e': {'somewhere', 'sometime', 'someone'}
NOTE: wildcard matches for 'some*e': {'somewhere', 'sometime', 'someone'}


## Spell-checking with a k-gram index

In [14]:
Jaccard_threshold = .25

Implement spell-correction for a word that was not found in the index, using the k-gram index.

  * Compute the k-grams from the input word (remember to include the '\$' characters);
  * find the matching words for each k-gram;
  * for each candidate word, check the Jaccard coefficent between the input word and the candidate (remember to include the '\$' characters);
  * if it's above (greater or equal to) the provided threshold, add it to the list of

In [15]:
def spellcorrect(word):
    word_grams = set(window('$' + word + '$', 3))
    ### <assignment>
    matches = [kgram_index[x] for x in word_grams]
    word_match_count = len(matches)
    all_candidates = set().union(*matches)
    outs = []
    for candidate in all_candidates:
        candidate_grams = set(window('$' + candidate + '$', 3))
        candidate_match_count = len(candidate_grams)
        intersection_count = len(candidate_grams & word_grams)
        union_count = len(candidate_grams.union(word_grams))
        if intersection_count / union_count >= Jaccard_threshold:
            outs.append((candidate, intersection_count / union_count))
    outs = sorted(outs, key=lambda x: x[1], reverse=True)
    return [x[0] for x in outs]
    ### </assignment>

Tests:

In [16]:
set(spellcorrect('smoetime'))

{'Beforetime',
 'Betimes',
 'Sometime',
 'bedtime',
 'betime',
 'betimes',
 'lifetime',
 'mestime',
 'pastime',
 'ruttime',
 'sometime',
 'time',
 '“Sometime'}

In [17]:
len(set(spellcorrect('smoetime')))

13

Note that the assertions that follow assume a Jaccard threshold of .25.

In [18]:
assert(set(spellcorrect('smoetime')) == {'betime', 'lifetime', 'sometime', 'pastime', 'bedtime', 'time', 'Betimes', 'mestime', 'ruttime', '“Sometime', 'Sometime', 'betimes', 'Beforetime'})

You can try your code as part of the query processor, which already calls `spellcorrect` whenever a (non-negated) term cannot be found in the index.

In [19]:
# adapted to windows lexicographic ordering!
assert(execute_query('smoetime AND Romeo') == {38})

NOTE: spell-correcting term 'smoetime' because no document contains it
NOTE: spell-corrections with Jaccard_threshold for 'smoetime': ['betime', 'time', 'lifetime', 'Sometime', 'sometime', '“Sometime', 'Beforetime', 'Betimes', 'ruttime', 'mestime', 'bedtime', 'betimes', 'pastime']
